ollamaの起動

In [ ]:
!pip install -q pypdf

In [ ]:
# 1. 依存ツールのインストール
!apt-get update && apt-get install -y zstd pciutils

# 2. Ollamaのインストール
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
import time
import requests

# 共通設定（ここでモデル名を一括管理します）
# 使用したいモデル名（glm4, llama3, qwen2-vl など）をここに記述
MODEL_NAME = "qwen3-vl:4b"

In [ ]:
def setup_ollama_server(model_name):
    """サーバーを再起動し、指定されたモデルを準備する"""
    print("🔄 サーバーをリセット中...")
    !pkill ollama
    time.sleep(2)

    # サーバーをバックグラウンドで起動
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)

    print(f"⏳ モデル '{model_name}' を準備中...")
    subprocess.run(["ollama", "pull", model_name])

    # 接続確認
    try:
        res = requests.get("http://localhost:11434/api/tags")
        if res.status_code == 200:
            models = [m['name'] for m in res.json()['models']]
            print(f"✅ Ollama server is live! 利用可能モデル: {models}")
            return True
    except:
        print("❌ サーバーの起動に失敗しました。")
        return False

# 実行
setup_ollama_server(MODEL_NAME)

In [ ]:
import gradio as gr
import requests
import json
import base64
from pypdf import PdfReader # PDF解析用

def extract_text_from_file(file_path):
    """ファイルからテキストを抽出する"""
    if file_path.lower().endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif file_path.lower().endswith('.pdf'):
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    return ""

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def chat_with_model(message, history):
    user_text = message["text"]
    user_files = message["files"]
    
    model = MODEL_NAME if 'MODEL_NAME' in globals() else "qwen3-vl:4b"
    url = "http://localhost:11434/api/chat"

    extracted_contexts = []
    images_base64 = []

    # ファイルの仕分けと処理
    for f in user_files:
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            images_base64.append(encode_image_to_base64(f))
        elif f.lower().endswith(('.txt', '.pdf')):
            content = extract_text_from_file(f)
            if content:
                extracted_contexts.append(f"--- File: {f} ---\n{content}")

    # テキストファイルの内容をプロンプトに結合
    full_prompt = user_text
    if extracted_contexts:
        context_str = "\n\n".join(extracted_contexts)
        full_prompt = f"以下の資料を参考にして回答してください:\n\n{context_str}\n\n質問: {user_text}"

    # メッセージ構造の作成
    current_msg = {"role": "user", "content": full_prompt}
    if images_base64:
        current_msg["images"] = images_base64

    # 履歴の追加
    msgs = []
    for user_msg, ai_msg in history:
        # 履歴はテキストのみを保持（シンプル化のため）
        msgs.append({"role": "user", "content": user_msg})
        msgs.append({"role": "assistant", "content": ai_msg})
    
    msgs.append(current_msg)

    payload = {"model": model, "messages": msgs, "stream": False}

    try:
        response = requests.post(url, json=payload, timeout=180) # 長文対応でタイムアウトを延長
        response.raise_for_status()
        return response.json()['message']['content']
    except Exception as e:
        return f"⚠️ エラー: {str(e)}"

# UIの設定
demo = gr.ChatInterface(
    fn=chat_with_model,
    title=f"🤖 {MODEL_NAME} Advanced Chat UI",
    description="画像、テキスト、PDFをアップロードして質問できます。",
    multimodal=True,
    textbox=gr.MultimodalTextbox(file_types=["image", ".txt", ".pdf"]),
    theme="soft"
)

demo.launch(share=True, debug=True)

In [ ]:
# def launch_vscode_tunnel():
#     """VS Code Tunnelを起動し、GitHub認証URLを表示する"""
#     import os

#     # ファイルが存在しない場合のみダウンロード
#     if not os.path.exists("code"):
#         print("⏳ VS Code CLI をダウンロード中...")
#         !curl -Lk 'https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64' --output vscode_cli.tar.gz
#         !tar -xf vscode_cli.tar.gz

#     print("\n" + "="*50)
#     print("🚀 VS Code Tunnel を開始します...")
#     print("※ URLが表示されるまで 30秒〜1分ほどかかる場合があります。")
#     print("="*50 + "\n")

#     # 認証情報をリセットして確実に新しいリンクを出す
#     # 以前の `--provider` エラーを避けるため、最小限の引数で実行します
#     !./code tunnel user logout
#     !./code tunnel --accept-server-license-terms

In [ ]:
# !pkill code  # 残っているVS Codeプロセスを強制終了